Imports

In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from PIL import Image

# Part 0: Load in pneumonia/healthy data to train CNN on

We will be training a CNN with a large pneumonia/healthy chest x-ray dataset and use transfer learning to train it for COVID/Non-COVID

In [2]:
init_d = {'filename': [], 'target': []}

# format: (file location, healthy/pneumonia(0/1))
files = [('train/Normal', 0), 
         ('train/Pneumonia', 1),
         ('test/Normal', 0), 
         ('test/Pneumonia', 1),
         ('val/Normal', 0), 
         ('val/Pneumonia', 1),]

for file in files:
    dir_path = os.path.abspath(os.getcwd())
    dir_path = dir_path + "/chest_xray/" + file[0]
    list = os.listdir(dir_path)
    for pic in list:
        if '.DS_Store' not in pic:
            init_d['filename'].append(pic)
            init_d['target'].append(file[1])

# Part 1: Preprocessing for images
Load in images for the corresponding target data with PIL. Reformat all images to the same dimensions and convert them to a numpy array for the models to use (where each is a (# pixels, # pixels) array).

In [3]:
# Load in image data from filenames with PIL
img_dim = 128
def normalize_images(d, curr_dir):
    """normalize all images inside dictionary"""
    images = np.empty((len(d['target']), img_dim, img_dim, 2))

    for i in range(len(d['filename'])):
        image = Image.open(curr_dir + d['filename'][i])

        # resize image shape
        image = image.resize((img_dim, img_dim))

        # convert to grayscale
        image = image.convert('LA')

        images[i] = np.array(image)

    # normalize values
    images = images / 255.0
    return images

In [4]:
# get train/test/val images
images = normalize_images(init_d, "chest_xray/pooled_imgs/")

# Part 2: Train Pneumonia/Healthy Classifier with CNN

In [20]:
# split data
x_train, x_test, y_train, y_test = train_test_split(images, init_d['target'], test_size=0.25, random_state=0)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [21]:
# build model for convolutional network
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_dim, img_dim, 2)))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# add more layers
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(64, activation='relu'))
cnn_model.add(layers.Dense(2))

# compile 
cnn_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 126, 126, 32)      608       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [22]:
history = cnn_model.fit(x_train, y_train, epochs=10,
                       validation_data=(x_test, y_test))

Epoch 1/10
138/138 [==============================] - 45s 325ms/step - loss: 0.2925 - accuracy: 0.8818 - val_loss: 0.2860 - val_accuracy: 0.8736
Epoch 2/10
138/138 [==============================] - 49s 352ms/step - loss: 0.1884 - accuracy: 0.9308 - val_loss: 0.1790 - val_accuracy: 0.9385
Epoch 3/10
138/138 [==============================] - 51s 370ms/step - loss: 0.1623 - accuracy: 0.9385 - val_loss: 0.1967 - val_accuracy: 0.9392
Epoch 4/10
138/138 [==============================] - 47s 342ms/step - loss: 0.1507 - accuracy: 0.9447 - val_loss: 0.1613 - val_accuracy: 0.9372
Epoch 5/10
138/138 [==============================] - 47s 338ms/step - loss: 0.1382 - accuracy: 0.9495 - val_loss: 0.1463 - val_accuracy: 0.9508
Epoch 6/10
138/138 [==============================] - 50s 364ms/step - loss: 0.1325 - accuracy: 0.9474 - val_loss: 0.1396 - val_accuracy: 0.9447
Epoch 7/10
138/138 [==============================] - 47s 340ms/step - loss: 0.1208 - accuracy: 0.9570 - val_loss: 0.1576 - val_ac

# Part 3: Load in COVID Data

Read in csv of metadata for the images and select target data. For our target data we are selecting COVID/Non-COVID, Survival, Intubation, and Admission to the ICU - resulting in 28 classes where class 0 = Non-COVID and class > 0 is COVID and some unique combination of the classes above. For the initial training and testing of this data, any class > 0 we set to 1 to train a binary classifier for COVID/Non-COVID

In [9]:
data = []
with open("metadata.csv") as file:
    for row in csv.reader(file):
        data.append(row)

data = data[1:]

d = {'filename': [], 'target': []}

for image in data:
    # skip volume data
    if image[22] == "volumes":
        continue
    
    # ignore CT scan data
    if image[19] == "CT":
        continue
    
    diagnosis = image[4].split("/")
    diagnosis = diagnosis[len(diagnosis) - 1].lower()
    # set target based on meta data
    if diagnosis != "covid-19":
        if diagnosis == "todo":
            # if not classified yet target = -1
            # d['filename'].append(image[23])
            # d['target'].append(-1)
            # ignore unknown data
            continue
        else:
            # if not COVID-19 target = 0
            d['filename'].append(image[23])
            d['target'].append(0)
    else:
        # if it is COVID-19
        if image[6] == "Y":
            # if they survived
            if image[7] == "Y":
                # if they were intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(1)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(2)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(3)
            elif image[7] == "N":
                # if they were not intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(4)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(5)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(6)
            else:
                # if they were intubated is unknown
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(7)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(8)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(9)
        elif image[6] == "N":
            # if they did not survive
            if image[7] == "Y":
                # if they were intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(10)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(11)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(12)
            elif image[7] == "N":
                # if they were not intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(13)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(14)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(15)
            else:
                # if they were intubated is unknown
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(16)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(17)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(18)
        else:
            # if their survival is unknown
            if image[7] == "Y":
                # if they were intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(19)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(20)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(21)
            elif image[7] == "N":
                # if they were not intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(22)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(23)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(23)
            else:
                # if they were intubated is unknown
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(24)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(25)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(26)
total = [0] * 28
for target in d['target']:
    if target == -1:
        total[27] += 1
    else:
        total[target] += 1

# set up binary classification (covid vs non-covid)
# all targets > 0 are covid
for i in range(len(d['target'])):
    if d['target'][i] > 0:
        d['target'][i] = 1

print(total)
#print(d)

file.close()


[279, 4, 3, 0, 0, 48, 0, 3, 6, 79, 8, 8, 2, 0, 1, 1, 0, 0, 18, 45, 31, 8, 0, 9, 0, 5, 225, 0]


In [10]:
# preprocess covid/noncovid images
covid_images = normalize_images(d, "model_images/")

# Part 2: Train COVID/Non-COVID with Transfer Learning

In [11]:
# split data
x_train, x_test, y_train, y_test = train_test_split(covid_images, d['target'], test_size=0.25, random_state=0)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [25]:
# get weights from outer model
all_weights = []
for layer in cnn_model.layers:
   w = layer.get_weights()
   all_weights.append(w)


In [38]:
temp_model = cnn_model

temp_model.layers.pop()

covid_model = temp_model

covid_model.add(layers.Dense(2))

covid_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

In [39]:
covid_history = covid_model.fit(x_train, y_train, epochs=10,
                              validation_data=(x_test, y_test))

Epoch 1/10
138/138 [==============================] - 46s 333ms/step - loss: 0.1049 - accuracy: 0.9620 - val_loss: 0.1631 - val_accuracy: 0.9501
Epoch 2/10
138/138 [==============================] - 45s 325ms/step - loss: 0.0768 - accuracy: 0.9695 - val_loss: 0.1622 - val_accuracy: 0.9488
Epoch 3/10
138/138 [==============================] - 43s 313ms/step - loss: 0.0733 - accuracy: 0.9715 - val_loss: 0.1612 - val_accuracy: 0.9495
Epoch 4/10
138/138 [==============================] - 51s 370ms/step - loss: 0.0582 - accuracy: 0.9781 - val_loss: 0.2484 - val_accuracy: 0.9460
Epoch 5/10
138/138 [==============================] - 54s 388ms/step - loss: 0.0465 - accuracy: 0.9827 - val_loss: 0.2386 - val_accuracy: 0.9242
Epoch 6/10
138/138 [==============================] - 50s 365ms/step - loss: 0.0558 - accuracy: 0.9795 - val_loss: 0.1711 - val_accuracy: 0.9515
Epoch 7/10
138/138 [==============================] - 37s 268ms/step - loss: 0.0327 - accuracy: 0.9872 - val_loss: 0.2401 - val_ac